In [1]:
import numpy as np
from random import gauss

path='C:\\Users\\15295\\Desktop\\量化金融\\financepy-master\\part1\\ch07'

a=[gauss(1.5,2) for i in range(1000000)]

In [2]:
import pickle
pkl_file=open(path+'data.pkl','wb')

%time pickle.dump(a,pkl_file)

Wall time: 33 ms


In [3]:
import sqlite3 as sq3

query='CREATE TABLE numbs (Date date, No1 real, No2 real)'
con=sq3.connect(path+'numbs.db')

con.execute(query)




OperationalError: table numbs already exists

In [ ]:
con.commit()

In [ ]:
import datetime as dt
con.execute('INSERT INTO numbs VALUES(?,?,?)',
           (dt.datetime.now(),0.12,7.3))

In [ ]:
data=np.random.standard_normal((10000,2)).round(5)
for row in data:
    con.execute('INSERT INTO numbs VALUES(?,?,?)',
               (dt.datetime.now(),row[0],row[1]))
con.commit()

In [ ]:
con.execute('SELECT * FROM numbs').fetchmany(10)

In [ ]:
pointer=con.execute('SELECT * FROM numbs')
for i in range(3):
    print(pointer.fetchone())

In [ ]:
con.close()

In [1]:
import tables as tb

filename='C:\\Users\\15295\\Desktop\\量化金融\\financepy-master\\part1\\ch07'+'tab.h5'

h5=tb.open_file(filename,'w')

rows=20000000

row_des={
    'Date':tb.StringCol(26,pos=1),
    'No1':tb.IntCol(pos=2),
    'No2':tb.IntCol(pos=3),
    'No3':tb.Float64Col(pos=4),
    'No4':tb.Float64Col(pos=5)
}

filters=tb.Filters(complevel=0)
tab=h5.create_table('/','ints_floats',row_des,title='Integers and Floats',
                  expectedrows=rows,filters=filters)


In [2]:
tab

/ints_floats (Table(0,)) 'Integers and Floats'
  description := {
  "Date": StringCol(itemsize=26, shape=(), dflt=b'', pos=0),
  "No1": Int32Col(shape=(), dflt=0, pos=1),
  "No2": Int32Col(shape=(), dflt=0, pos=2),
  "No3": Float64Col(shape=(), dflt=0.0, pos=3),
  "No4": Float64Col(shape=(), dflt=0.0, pos=4)}
  byteorder := 'little'
  chunkshape := (5242,)

In [3]:
import numpy as np
import datetime as dt
ran_int=np.random.randint(0,10000,size=(rows,2))
ran_flo=np.random.standard_normal((rows,2)).round(5)

In [4]:
pointer=tab.row

In [5]:
%time 
for i in range(rows):
    pointer['Date']=dt.datetime.now()
    pointer['No1']=ran_int[i,0]
    pointer['No2']=ran_int[i,1]
    pointer['No3']=ran_flo[i,0]
    pointer['No4']=ran_flo[i,1]
tab.flush()

Wall time: 0 ns


In [7]:
dty=np.dtype([('Date','S26'),('No1','<i4'),('No2','<i4'),('No3','<f8'),('No4','<f8')])
sarray=np.zeros(len(ran_int),dtype=dty)
